# Individual Feature Determination

## Preparation

### Import

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.set_option('display.max_columns', 100)

## Function Definition

### Calculate Individual Features

#### Percentage Members Shown Face

In [ ]:
def members_show_face_percen(df_group,df_indiv):

  grouped = df_indiv.groupby('group')
  dict_shown_face = {}

  for group, data in grouped:
      count = data['shown_face'].value_counts().get(True, 0)
      dict_shown_face[group] = count
  
  # Fill the DataFrame with the values from the dictionary
  df_group['members_show_face'] = df_group.index.map(dict_shown_face).fillna(0)
  df_group['members_show_face_percen'] = df_group['members_show_face'] / df_group['num_members']
  df_group= df_group.drop('members_show_face', axis=1)

  return df_group

#### Percentage Share Screen

In [ ]:
def screen_share_percen(df_group):

  df_group['screen_share_percen'] = df_group['share_screen_time'] / df_group['total_duration_recorded']
  	
  return df_group


#### Overall Duration Spoken

In [ ]:
def total_duration_spoken(df_group,df_indiv):

  grouped = df_indiv.groupby('group')
  spoken_duration = {}

  for group, data in grouped:
      sum = data['indiv_spoken_time'].sum()
      spoken_duration[group] = sum

  df_group['total_duration_spoken'] = df_group.index.map(spoken_duration).fillna(np.NaN)

  return df_group


#### Ratio Spoken-Recorded

In [ ]:
def ratio_spoken_recorded(df_group,df_indiv):

  df_group['ratio_spoken_recorded'] = df_group['total_duration_spoken'] / df_group['total_duration_recorded']

  return df_group


#### Average Meeting Turn Duration

In [ ]:
def avg_meeting_turn_duration(df_group, df_indiv):
  
    grouped = df_indiv.groupby('group')
    avg_durations = {}

    for group, data in grouped:
        turns_and_avg = []
        total = 0
        for index, row in data.iterrows():
            if pd.notna(row['num_turns']) and pd.notna(row['average_turn_duration']):
                turns_and_avg.append((row['num_turns'], row['average_turn_duration']))
                total += row['num_turns']
        if turns_and_avg:
            result = sum(pair[0] / total * pair[1] for pair in turns_and_avg)
            avg_durations[group] = result
    
    df_group['avg_meeting_turn_duration'] = df_group.index.map(avg_durations).fillna(np.NaN)

    return df_group

#### Total Turns Overall

In [ ]:
def total_turns(df_group,df_indiv):

  grouped = df_indiv.groupby('group')
  turns = {}

  for group, data in grouped:
      sum = data['num_turns'].sum()
      turns[group] = sum

  df_group['total_turns'] = df_group.index.map(turns).fillna(np.NaN)

  return df_group


#### Turns per minute

In [ ]:
def turns_in_a_min(df_group,df_indiv):

  df_group['turns_in_a_min'] = df_group['total_turns'] / df_group['total_duration_recorded']*60

  return df_group


#### Average meeting Words in Message

In [ ]:
def average_meeting_words_message(df_group,df_indiv):

    grouped = df_indiv.groupby('group')
    avg_words = {}

    for group, data in grouped:
        words_and_avg = []
        total = 0
        for index, row in data.iterrows():
            if pd.notna(row['num_words']) and pd.notna(row['avg_words_turn']):
                words_and_avg.append((row['num_words'], row['avg_words_turn']))
                total += row['num_words']
        if words_and_avg:
            result = sum(pair[0] / total * pair[1] for pair in words_and_avg)
            avg_words[group] = result
    
    df_group['average_meeting_words_message'] = df_group.index.map(avg_words).fillna(np.NaN)

    return df_group

#### Total Words in General

In [ ]:
def total_words(df_group,df_indiv):

  grouped = df_indiv.groupby('group')
  words = {}

  for group, data in grouped:
      sum = data['num_words'].sum()
      words[group] = sum

  df_group['total_words'] = df_group.index.map(words).fillna(np.NaN)

  return df_group


#### Words per minute

In [ ]:
def words_in_a_min(df_group,df_indiv):

  df_group['words_in_a_min'] = df_group['total_words'] / df_group['total_duration_recorded']*60

  return df_group

### Calculate Descriptive Estadistics given a column

In [ ]:
def calc_descriptive_stadistics_column2(df_group, df_indiv, group_col, column):

    grouped = df_indiv.groupby(group_col)
    turns = {}
    statistics_list= ['avg', 'stdev', 'min', 'max', 'diff']
    blank_dictionaries = {i: {} for i in statistics_list}

    for group, data in grouped:
      blank_dictionaries['avg'][group] = data[column].mean()
      blank_dictionaries['stdev'][group] = data[column].std()
      blank_dictionaries['min'][group] = data[column].min()
      blank_dictionaries['max'][group] = data[column].max()
      blank_dictionaries['diff'][group] = data[column].max() - data[column].min()

    for stats in statistics_list:
      df_group[stats+"_"+column] = blank_dictionaries[stats].values

    # Return the modified DataFrame
    return df_group

In [ ]:
def calc_descriptive_statistics_column(df_group, df_indiv, group_col, column):
    grouped = df_indiv.groupby(group_col)
    statistics_list = ['avg', 'stdev', 'min', 'max', 'diff']
    blank_dictionaries = {i: {} for i in statistics_list}

    for group, data in grouped:
        blank_dictionaries['avg'][group] = data[column].mean()
        blank_dictionaries['stdev'][group] = data[column].std()
        blank_dictionaries['min'][group] = data[column].min()
        blank_dictionaries['max'][group] = data[column].max()
        blank_dictionaries['diff'][group] = data[column].max() - data[column].min()

    new_data = pd.DataFrame.from_dict(blank_dictionaries)

    # Rename the columns in new_data
    new_data.columns = [stats + "_" + column for stats in statistics_list]

    # Merge the new DataFrame with df_group
    df_group = pd.concat([df_group, new_data], axis=1)

    # Return the modified DataFrame
    return df_group


### Add Descriptive Estadistics given a Range

In [ ]:
def add_descriptive_stadistics_range(df_group, df_indiv, start_col, end_col):

  cols_range = df_indiv.columns[df_indiv.columns.get_loc(start_col):df_indiv.columns.get_loc(end_col)+1].tolist()
    
  # Loop through the emotion columns and calculate the average for each
  for col in cols_range:
      df_group= calc_descriptive_statistics_column(df_group, df_indiv, "group", col)

  return df_group

### Calculate Group Features

In [ ]:
def calc_group_features(indiv_features_file, group_features, file_out):

  df_indiv = pd.read_excel(indiv_features_file, index_col=0)
  df_group = pd.read_excel(group_features_file, index_col=0)
  df_group = df_group.transpose()

  stripped_rows = df_group.iloc[:4]  # Extract the first 4 rows and store them
  df_group = df_group.iloc[4:]
  df_indiv = df_indiv.iloc[3:]

  df_group= members_show_face_percen(df_group,df_indiv)
  df_group= screen_share_percen(df_group)
  df_group= total_duration_spoken(df_group,df_indiv)
  df_group= ratio_spoken_recorded(df_group,df_indiv)
  df_group= avg_meeting_turn_duration(df_group,df_indiv)
  df_group= total_turns(df_group,df_indiv)
  df_group= turns_in_a_min(df_group,df_indiv)
  df_group= average_meeting_words_message(df_group,df_indiv)
  df_group= total_words(df_group,df_indiv)
  df_group= words_in_a_min(df_group,df_indiv)
  
  df_group=add_descriptive_stadistics_range(df_group, df_indiv,"indiv_spoken_time", "text_sadness")
  
  #df_group = df_group.transpose()
  df_group = pd.concat([stripped_rows, df_group]).reset_index(drop=True)
  df_group.to_csv(file_out,encoding='utf-8-sig', index=False)

  return df_group

## Use of Function

In [ ]:
indiv_features_file = r'/content/drive/MyDrive/Projects/tps/data/12. features/4_individual_features_final_ratio.xlsx'
group_features_file = r'/content/drive/MyDrive/Projects/tps/data/12. features/6_group_meetings_features_byhand_filled.xlsx'
file_out= r'/content/drive/MyDrive/Projects/tps/data/12. features/7_group_meetings_features_final.xlsx'

In [ ]:
calc_group_features(indiv_features_file, group_features_file, file_out)


,group_name,num_members,num_recordings,total_duration_recorded,share_screen_time,members_show_face_percen,screen_share_percen,total_duration_spoken,ratio_spoken_recorded,avg_meeting_turn_duration,total_turns,turns_in_a_min,average_meeting_words_message,total_words,words_in_a_min,avg_indiv_spoken_time,stdev_indiv_spoken_time,min_indiv_spoken_time,max_indiv_spoken_time,diff_indiv_spoken_time,avg_indiv_spoken_time_ratio,stdev_indiv_spoken_time_ratio,min_indiv_spoken_time_ratio,max_indiv_spoken_time_ratio,diff_indiv_spoken_time_ratio,avg_average_turn_duration,stdev_average_turn_duration,min_average_turn_duration,max_average_turn_duration,diff_average_turn_duration,avg_average_turn_duration_ratio,stdev_average_turn_duration_ratio,min_average_turn_duration_ratio,max_average_turn_duration_ratio,diff_average_turn_duration_ratio,avg_avg_time_without_speaking,stdev_avg_time_without_speaking,min_avg_time_without_speaking,max_avg_time_without_speaking,diff_avg_time_without_speaking,avg_avg_time_without_speaking_ratio,stdev_avg_time_without_speaking_ratio,min_avg_time_without_speaking_ratio,max_avg_time_without_speaking_ratio,diff_avg_time_without_speaking_ratio,avg_max_time_without_speaking,stdev_max_time_without_speaking,min_max_time_without_speaking,max_max_time_without_speaking,diff_max_time_without_speaking,...,avg_max_words_turn,stdev_max_words_turn,min_max_words_turn,max_max_words_turn,diff_max_words_turn,avg_max_words_turn_ratio,stdev_max_words_turn_ratio,min_max_words_turn_ratio,max_max_words_turn_ratio,diff_max_words_turn_ratio,avg_speech_neu,stdev_speech_neu,min_speech_neu,max_speech_neu,diff_speech_neu,avg_speech_ang,stdev_speech_ang,min_speech_ang,max_speech_ang,diff_speech_ang,avg_speech_hap,stdev_speech_hap,min_speech_hap,max_speech_hap,diff_speech_hap,avg_speech_sad,stdev_speech_sad,min_speech_sad,max_speech_sad,diff_speech_sad,avg_text_joy,stdev_text_joy,min_text_joy,max_text_joy,diff_text_joy,avg_text_anger,stdev_text_anger,min_text_anger,max_text_anger,diff_text_anger,avg_text_fear,stdev_text_fear,min_text_fear,max_text_fear,diff_text_fear,avg_text_sadness,stdev_text_sadness,min_text_sadness,max_text_sadness,diff_text_sadness
0,string,int,int,int (seg),int (seg),float,float,int(seg),float,float,int,float,float,int,float,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,input,input,input,input,Σ len(sec) if\nscreen_share=True,count (shown_face==true)/\nnum_members,time_share_screen /\ntotal_duration_recorded,Σ indiv_time_spoken,total_duration_spoken/\ntotal_duration_recorded,Σ each probability *\naverage_turn_duration,Σ num_turns,total_turns * 60\n/ total_duration_recorded,num_words/ num_turns,Σ avg_words_turn*\neach probability,total_words * 60\n/ total_duration_recorded,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,name of\nthe group,number of\nintegrants,number of\nrecorded meetings,duration of all\nof the recordings,duration of\nscreen share,percentage of members\nthat turn on camera,ratio between the\nscreen has ben shared,duration of the overall\nspoken time,ratio between the spoken\nand recorded time,most common duration\nof a turn in the convers...,total turns in\nthe conversation,how many turns are\nin a minute,most common number of\nwords in a meeting message,total words in\nthe conversation,how many words are\nin a minute,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na